# Model builder

In [20]:
import numpy as np
import math

from sacio import write_sac_ascii

In [8]:
#loading default values
#turn this to a function

#simparams
simparams={}

simparams["boundary_conditions"]=[2.0, 2.0, 2.0, 2.0, 2.0, 2.0]
simparams["cosmological_simulation"]=0
simparams["current_iteration"]=0
simparams["current_time"]=0
simparams["dimensionality"]=3
simparams["domain_dimensions"]=[16,16,16]
simparams["domain_left_edge"]=[0,0, 0.0]
simparams["domain_right_edge"]=[0, 0, 0]
simparams["eta"]=0.0
simparams["adiab"]=1.0
simparams["field_ordering"]=1
simparams["gamma"]=1.66666667
simparams["gravity0"]=0.0
simparams["gravity1"]=0.0
simparams["gravity2"]=0.0
simparams["nu"]=0.0
simparams["num_ghost_zones"]=0
simparams["refine_by"]=0
simparams["unique_identifier"]='notset'
simparams["nw"]=13  # %number of fields
simparams["neqpar"]=7 # %number of equation parameters

#simgridinfo
simgridinfo={}
simgridinfo["ndimensions"]=3
#%grid_dimensions=64*ones(3,1);
simgridinfo["grid_dimensions"]=[128,128,128]
simgridinfo["grid_left_index"]=[1,1,1]
simgridinfo["grid_right_index"]=[0,0,0]
simgridinfo["grid_level"]=0
simgridinfo["grid_parent_id"]=0
simgridinfo["grid_particle_count"]=0
simgridinfo["data_software"]='matsac'
simgridinfo["data_software_version"]='0.1'

#simdata
simdata={}
simdata["simparams"]=simparams
simdata["gridinfo"]=simgridinfo
        
gds=simgridinfo["grid_dimensions"]
simdata["w"]=np.zeros((gds[0],gds[1],gds[2],simgridinfo["ndimensions"]+simparams["nw"]))




In [11]:


newfilename='3D_128_4Mm_asc.ini';
#%newfilename='/data/cs1mkg/smaug/configs/3D_128_4Mm_bin.ini';

consts={}
consts["mu"]=0.6e0 #mu_therm
consts["R"]=8.31e3
consts["fgamma"]=1.66666667e0
consts["ggg"]=-274.0e0 #% acceleration due to gravity on the sun
consts["mu"]=4*math.pi/1.0e7 #magnetic permeability


ngx1=2
ngx2=2
ngx3=2

it=0
time=0
ndim=3
neqpar=7
nw=13
nx1=128
nx2=128
nx3=128


gamma=1.666667
adiab=1.0
eta=0
g1=-274
g2=0
g3=0
       
xmin=133333.33
ymin=1953.1
zmin=1953.1
xmax=5955555.6e0
ymax=4.0e6
zmax=4.0e6
       
dx=(xmax-xmin)/(nx1-2*ngx1)
dy=(ymax-ymin)/(nx2-2*ngx2)
dz=(zmax-zmin)/(nx3-2*ngx3)
       
       
xx=np.zeros((nx1,nx2,nx3))
yy=np.zeros((nx1,nx2,nx3))
zz=np.zeros((nx1,nx2,nx3))

rheight=np.zeros(nx1)
       
       
for i in range(nx1):
    for j in range(nx2):
        for k in range(nx3):
            xx[i,j,k]=(xmin-ngx1*dx)+i*dx
            rheight[i]=(xmin-ngx1*dx)+i*dx
            yy[i,j,k]=(ymin-ngx1*dx)+dy*j
            zz[i,j,k]=(zmin-ngx1*dx)+dz*k
      
simdata["w"]=np.zeros((nx1,nx2,nx3,nw))




simparams["current_iteration"]=it
simparams["current_time"]=time
simparams["dimensionality"]=ndim
simparams["domain_dimensions"]=[nx1,nx2,nx3]
#         simparams.domain_left_edge=[0;0; 0.0];
#         simparams.domain_right_edge=[0; 0; 0];
simparams["eta"]=eta
simparams["adiab"]=adiab
#        field_ordering=1;
simparams["gamma"]=gamma
simparams["gravity0"]=g1
simparams["gravity1"]=g2
simparams["gravity2"]=g3
        
        
        
        
simparams["domain_left_edge"]=[xx[0,0,0],yy[0,0,0],zz[0,0,0]]      
simparams["domain_right_edge"]=[xx[nx1-1,0,0],yy[0,nx2-1,0],zz[0,0,nx3-1]]


        
simgridinfo["grid_dimensions"]=[nx1,nx2,nx3]


In [12]:
#set up the magnetic field generation parameters
Bmax=0.005
Bmin=0.0002
z_shift=0.0
x_shift=0.0
y_shift=0.0

A=0.5
sscale=1.0e6
b0z_top=0.08

f0=2.0e6  #tube opening parameter
ab0z=20.0 #  bz amplitude
r2=0.5e6 #tube radius




In [13]:
#parameters for setting up atmosphere
pars={}
pars["ytr"]=2.0e6
pars["wtr"]=0.1e6
pars["tc"]=1.8e6
pars["tch"]=8000

In [ ]:
#parameters setting up hydrostatic density
rho0=1.17e-4  #rho0=
p0=2.7865e-4  #p0=

# Generate parametric model

In [1]:
#temperature generation function
#generate the temperature at a height
def temp(h,pars):
    tmptemp=1+math.tanh((h-pars["ytr"])/pars["wtr"])
    tmptemp=pars["tch"]+((pars["tc"]-pars["tch"])/2.0)*tmptemp
    return tmptemp

def hydrodens(h, ii, nx1, deltah,rtemp,consts,rho0):
    t0=rtemp[0]
    rsum=0
    if ii==0:
        hscale=-consts["R"]*t0/(consts["mutherm"]*consts["ggg"])
        rsum=rsum+deltah/hscale
    else:
        for i in range(ii+1):
            hscale=-consts["R"]*rtemp[i]/(consts["mutherm"]*consts["ggg"])
            rsum=rsum+deltah/hscale
        
    dens=rsum*(t0/rtemp[ii])*math.exp(-rsum)    
    return dens

def hydropres():
    return


In [16]:
#calculate heights and temperatures
rtemp=np.zeros(nx1)
i=0
for h in rheight:
    rtemp[i]=temp(h,pars)
    i=i+1

In [ ]:
#calculate hydrostatically balanced density hydrodens
i=0
t0=rtemp[0]
ld=np.zeros(nx1)
for h in rheight:
    ld(i)=hydrodens(h,i,nx1,deltah,rtemp,consts)
    i=i+1

In [ ]:
#calculate hydrostatically balance pressure

In [ ]:
# calculate energy
energg=pres/(consts["fgamma"]-1)

In [ ]:
#rebuild the model loop over i,j,k set simdata["w"]

In [ ]:
#generate static field

In [ ]:
#apply hydrostatic balance

In [ ]:
#write the sac3d output

#set modelinfo
#    header=modelinfo[0]
#    dim=modelinfo[6]
#    ndim=modelinfo[3]
#    nfields=modelinfo[5]
#    time=modelinfo[2]
#    nits=modelinfo[1]
#    nvar=modelinfo[4]
modelinfo=[]
modelinfo.append('Header info')


write_sac_ascii(newfilename, simdata["w"], modelinfo)